<a href="https://colab.research.google.com/github/sosodoit/yolov5/blob/main/YOLOv5_Task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<ul>Task 0 (YOLOv5 note)
<dl>용어, 개념, 궁금한 문제들 기록 (<a href="https://colab.research.google.com/drive/1u8S1vF5g9dSR8oDBJxjqiY9AH0C43jjF?usp=sharing">link</a>)</dl>
</ul>

<ol type="I">
<strong>
  <li>Task 1 (사전작업)</li>
  <dl>YOLOv5 모델작업에 들어가기전, 데이터를 확인하고 정제한 작업 내용을 담고 있습니다(here)</dl>
  <ul type="square">
  <li>Putting data into a folder</li>
  <li>Convert the Annotations into the YOLOv5 Format</li>
  <li>Testing the annotations</li>
  <li>Partition the Dataset</li>
  </ul>
</strong>
  <li>Task 2 (학습)</li>
  <dl>YOLOv5 환경설정부터 학습한 작업내용을 담고 있습니다(<a href="https://colab.research.google.com/drive/1zHrTaAlsT4tDxZhGhUYK1ZIGjtrrvv-y?usp=sharing">link</a>)</dl>
  <ul type="square">
  <li>Set up the environment</li>
  <li>Data Config File</li>
  <li>Hyper-parameter Config File</li>
  <li>Custom Network Architecture</li>
  <li>Train the Model</li>
  </ul>
  <li>Task 3 (결과)</li>
  <dl>학습 후 테스트와 성능평가의 결과내용을 담고 있습니다(<a href="https://colab.research.google.com/drive/1---iUi0KXpVVjn95BNmsDyScnzLmFY6p?usp=sharing">link</a>)</dl>
  <ul type="square">
  <li>Test</li>
  <li>Computing the mAP on test dataset</li>
  </ul>
</ol>

---
# 수정된 작업(up-to-date)

**1. Putting data into a folder**
<ul>나눠진 데이터를 작업할 폴더에 images와 labels폴더 아래로 모음</ul>

**Original Data**

- [원천]도심로_주간일출_맑음_45_전방, [라벨]도심로_주간일출_맑음_45_전방
- [원천]도심로_주간일출_맑음_60_전방, [라벨]도심로_주간일출_맑음_60_전방
- [원천]도심로_주간일출_맑음_120_전방, [라벨]도심로_주간일출_맑음_120_전방
<dl>총 2318개의 이미지와 라벨링된 xml</dl>

**Datafile Directory**

- dataset
  - images
    - train
    - val
  - labels
    - train
    - val

**2. Convert the Annotations into the YOLOv5 Format**

주의: dataset폴더가 있는 상위폴더 경로에서 진행해야함

``` python
#ex
%cd /content/gdrive/MyDrive/객체탐지
!ls
# dataset
```


In [ ]:
############ 경로설정 ############
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/객체탐지
!ls

In [ ]:
############ xml 파일 확인 ############
#!cat dataset/labels/Bright_20130218_153731_000210_v001_1.xml

In [ ]:
############ XML파일에서 정보 긁어오는 함수 ############
def extract_info_from_xml(xml_file):
    root = ET.parse(xml_file).getroot()
    
    info_dict = {}
    info_dict['bboxes'] = []

    for elem in root:
        if elem.tag == "filename":
            info_dict['filename'] = elem.text
            
        elif elem.tag == "size":
            image_size = []
            for subelem in elem:
                image_size.append(int(subelem.text))
            
            info_dict['image_size'] = tuple(image_size)
        
        elif elem.tag == "object":
            bbox = {}
            for subelem in elem:
                if subelem.tag == "name":
                    bbox["class"] = subelem.text
                    
                elif subelem.tag == "bndbox":
                    for subsubelem in subelem:
                        bbox[subsubelem.tag] = int(subsubelem.text)            
            info_dict['bboxes'].append(bbox)
    
    return info_dict

In [ ]:
############ 긁어온 xml 정보 확인 ############
print(extract_info_from_xml('dataset/labels/Bright_20130218_153731_000210_v001_1.xml'))

In [ ]:
############ 클래스명과 클래스ID dictionary 만듬 ############
class_name_to_id_mapping = {"Vehicle_Car": 0,
                           "Vehicle_Bus": 1,
                           "Vehicle_Motorcycle": 2,
                           "Vehicle_Unknown": 3}

In [ ]:
############ yolov5 형식으로 변환하는 함수 ############
def convert_to_yolov5(info_dict):
    global class_id
    print_buffer = []
    
    for b in info_dict["bboxes"]:
        try:
            class_id = class_name_to_id_mapping[b["class"]]
        except KeyError:
            print("Invalid Class. Must be one from ", class_name_to_id_mapping.keys())
        
        b_center_x = (b["xmin"] + b["xmax"]) / 2 
        b_center_y = (b["ymin"] + b["ymax"]) / 2
        b_width    = (b["xmax"] - b["xmin"])
        b_height   = (b["ymax"] - b["ymin"])
        
        image_w, image_h, image_c = info_dict["image_size"]  
        b_center_x /= image_w 
        b_center_y /= image_h 
        b_width    /= image_w 
        b_height   /= image_h 
        
        print_buffer.append("{} {:.3f} {:.3f} {:.3f} {:.3f}".format(class_id, b_center_x, b_center_y, b_width, b_height))
        
    save_file_name = os.path.join("dataset/labels", info_dict["filename"].replace("jpg", "txt"))
    
    print("\n".join(print_buffer), file= open(save_file_name, "w"))

In [ ]:
############ 함수실행(포맷)을 위해 xml 파일이름 가져오기 ############
annotations = [os.path.join('dataset/labels', x) for x in os.listdir('dataset/labels') if x[-3:] == "xml"]
annotations.sort()

############ 변환 후 저장(시간 조금 소요) ############
for ann in tqdm(annotations):
    info_dict = extract_info_from_xml(ann)
    convert_to_yolov5(info_dict)
annotations = [os.path.join('dataset/labels', x) for x in os.listdir('dataset/labels') if x[-3:] == "txt"]

**3. Testing the annotations**

xml이 잘 변환되어서 bb 잘 잡는지 테스트

In [ ]:
%cd /content/gdrive/MyDrive/객체탐지

In [ ]:
random.seed(0)

class_id_to_name_mapping = dict(zip(class_name_to_id_mapping.values(), class_name_to_id_mapping.keys()))

def plot_bounding_box(image, annotation_list):
    annotations = np.array(annotation_list)
    w, h = image.size
    
    plotted_image = ImageDraw.Draw(image)

    transformed_annotations = np.copy(annotations)
    transformed_annotations[:,[1,3]] = annotations[:,[1,3]] * w
    transformed_annotations[:,[2,4]] = annotations[:,[2,4]] * h 
    
    transformed_annotations[:,1] = transformed_annotations[:,1] - (transformed_annotations[:,3] / 2)
    transformed_annotations[:,2] = transformed_annotations[:,2] - (transformed_annotations[:,4] / 2)
    transformed_annotations[:,3] = transformed_annotations[:,1] + transformed_annotations[:,3]
    transformed_annotations[:,4] = transformed_annotations[:,2] + transformed_annotations[:,4]
    
    for ann in transformed_annotations:
        obj_cls, x0, y0, x1, y1 = ann
        plotted_image.rectangle(((x0,y0), (x1,y1)))
        
        plotted_image.text((x0, y0 - 10), class_id_to_name_mapping[(int(obj_cls))])
    
    plt.imshow(np.array(image))
    plt.show()

# Get any random annotation file 
annotation_file = random.choice(annotations)
with open(annotation_file, "r") as file:
    annotation_list = file.read().split("\n")[:-1]
    annotation_list = [x.split(" ") for x in annotation_list]
    annotation_list = [[float(y) for y in x ] for x in annotation_list]

#Get the corresponding image file
image_file = annotation_file.replace("labels", "images").replace("txt", "jpg")
assert os.path.exists(image_file)

#Load the image
image = Image.open(image_file)

#Plot the Bounding Box
plot_bounding_box(image, annotation_list)

**4. Partition the Dataset**

YOLOv5 학습 진행 하기전, train-valid dataset을 나누는 작업을 진행

In [ ]:
########## 사진과 라벨링 파일명 읽어오기 ##########
images = [os.path.join('dataset/images', x) for x in os.listdir('dataset/images')]
annotations = [os.path.join('dataset/labels', x) for x in os.listdir('dataset/labels') if x[-3:] == "txt"]

images.sort()
annotations.sort()

In [ ]:
########## train-valid 8:2 비율로 데이터셋 나누기 ##########
train_images, val_images, train_annotations, val_annotations = train_test_split(images, annotations, test_size = 0.2, random_state = 2000)

In [ ]:
########## 나눠진 데이터를 각각 train, val 폴더에 옮겨줌 ##########
!mkdir dataset/images/train dataset/images/val dataset/labels/train dataset/labels/val

def move_files_to_folder(list_of_files, destination_folder):
    for f in list_of_files:
        try:
            shutil.move(f, destination_folder)
        except:
            print(f)
            assert False

move_files_to_folder(train_images, 'dataset/images/train')
move_files_to_folder(val_images, 'dataset/images/val/')
move_files_to_folder(train_annotations, 'dataset/labels/train/')
move_files_to_folder(val_annotations, 'dataset/labels/val/')

---
# 과거작업

## [1] Rename

In [ ]:
# 1) 경로설정
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/객체탐지/dataset

In [ ]:
# 2) 필요한 라이브러리 임포트
import os
import torch
from IPython.display import Image
import os 
import random
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# 3) 이미지 폴더 접근
## 주어진 디렉토리에 있는 항목들의 이름을 담고 있는 리스트를 반환합니다.
## 리스트는 임의의 순서대로 나열됩니다.
images = os.listdir('origin/images')
images.sort()

labels = os.listdir('origin/labels')
labels.sort()

In [ ]:
# 파일명 확인
print(images[1200:1204])
print(labels[1200:1204])

['3_20201019_165355_000570.jpg', '3_20201019_165355_000600.jpg', '3_20201019_165355_000630.jpg', '3_20201019_171412_000360.jpg']
['3_20201019_165355_000570_v001_1.xml', '3_20201019_165355_000600_v001_1.xml', '3_20201019_165355_000630_v001_1.xml', '3_20201019_171412_000360_v001_1.xml']


In [ ]:
file_path = 'origin/images'
result_path = 'images'
file_names = images

file2_path = 'origin/labels'
result2_path = 'labels'
file2_names = labels

In [ ]:
# 4) 차례대로 파일 이름을 변경하고 저장 ".jpg"
## os.rename(src, dst) 메서드는 파일 또는 디렉토리(폴더) src의 이름을 dst로 변경합니다.
i = 1
for name in file_names:
    src = os.path.join(file_path, name)
    dst = str(i) + '.jpg'
    dst = os.path.join(result_path, dst)
    os.rename(src, dst)
    i += 1

In [ ]:
# 4) 차례대로 파일 이름을 변경하고 저장 ".xml"
## os.rename(src, dst) 메서드는 파일 또는 디렉토리(폴더) src의 이름을 dst로 변경합니다.
i = 1
for name in file2_names:
    src = os.path.join(file2_path, name)
    dst = str(i) + '.xml'
    dst = os.path.join(result2_path, dst)
    os.rename(src, dst)
    i += 1

## [2] XmlToTxt
- https://github.com/Isabek/XmlToTxt

## [3] Devide

## [4] 사진 거르기

class 1,2,3,4 하나라도 있으면 살리고 없으면 제거.

In [ ]:
# 1) 경로설정
%cd /content/gdrive/MyDrive/객체탐지/dataset

In [ ]:
# 2) 필요한 라이브러리 임포트
import os
from glob import glob

In [ ]:
img_list = glob('labels/train/*.txt')
val_img_list = glob('labels/val/*.txt')

print(len(img_list))
print(len(val_img_list))

In [ ]:
four = [] #체크를 위한 리스트 생성

## 여기서 val_img_list / img_list 두개를 변경하면서 내부의 다른 index를 제거해주어야 한다.
for x in img_list:
    four = [] #새로운 라벨링 데이터 .txt를 불렀을 때 초기화를 위함
    f = open(x, 'r')
    lines = f.readlines() 
    for line in lines: #읽어들인 모든 데이터를 한 줄씩 불러옴

        # 1,2,3,4 없으면 모두 제거하기 위해 check.
        # 1:Vehicle_Car 2:Vehicle_Bus 3:Vehicle_Motorcycle 4:Vehicle_Unknown
        if (int(line.split(' ')[0]) == 1) or (int(line.split(' ')[0]) == 2) or (int(line.split(' ')[0]) == 3) or (int(line.split(' ')[0]) == 4):
            four.append(line)

    # 만약 넷 중 하나도 없는 사진이 있다면 제거.
    if len(four) == 0:
        print("four 내용 없음")
        os.remove(x.split('.txt')[0].replace('labels','images')+'.jpg')
        os.remove(x)
        
    f.close()
    
    #두 개가 존재한다면 다시 두 개만 불러오도록 재정의.
    #다시 쓰는이유는 하나의 사진에 1,2,3,4 이외에 다른 값도 있는 사진을 수정하기 위함.
    #w를 이용해 재정의하면 원래 txt파일에 존재하는 모든 수치를 제거하고 처음부터 적는다. 이를 이용함.
    if len(four) >= 1:
        print("four 씀: ", len(four))
        w = open(x,'w')
        w.writelines(four)
        w.close()

In [ ]:
four = []
count1 = 0
count2 = 0
count3 = 0
count4 = 0

for x in val_img_list:
    four = []
    f = open(x, 'r')
    lines = f.readlines()

    for line in lines:
        if line.split(' ')[0] == '1':
            four.append(line.replace(line.split(' ')[0], '0'))
            count1 += 1
        elif line.split(' ')[0] == '2':
            four.append(line.replace(line.split(' ')[0], '1'))
            count2 += 1
        elif line.split(' ')[0] == '3':
            four.append(line.replace(line.split(' ')[0], '2'))
            count3 += 1
        elif line.split(' ')[0] == '4':
            four.append(line.replace(line.split(' ')[0], '3'))
            count4 += 1
  
    f.close()

    w = open(x,'w')
    w.writelines(four)
    w.close()

print(' 1:Vehicle_Car %d 개 변환 완료 ' % count1 )
print(' 2:Vehicle_Bus %d 개 변환 완료 ' % count2 )
print(' 3:Vehicle_Motorcycle %d 개 변환 완료 ' % count3 )
print(' 4:Vehicle_Unknown %d 개 변환 완료 ' % count4 )

## val_img_list
# 1:Vehicle_Car 2155 개 변환 완료 
# 2:Vehicle_Bus 109 개 변환 완료 
# 3:Vehicle_Motorcycle 195 개 변환 완료 
# 4:Vehicle_Unknown 579 개 변환 완료 

## img_list (train)
# 1:Vehicle_Car 8967 개 변환 완료 
# 2:Vehicle_Bus 399 개 변환 완료 
# 3:Vehicle_Motorcycle 790 개 변환 완료 
# 4:Vehicle_Unknown 2395 개 변환 완료 

# **Issue**

파일명이 복잡해서 간단하게 바꾼 후 구분할 클래스 4종이 하나라도 안들어 있는 사진이 있다면 거르는 작업을 통해 학습 성능을 높여 볼려고 생각하고 진행했다. 

이 때, 굉장히 이상하게 detection되고 있는 것을 확인하고 문제점을 찾아보니 파일명을 rename하는 과정에서 xml에 있는 filename과 rename된 파일이름이 달라지면서 학습과정에 문제가 발생했다. 또한, 걸러진 사진이 10개도 되지 않아서 정확도에 주는 영향은 미미했다.

이 작업을 훗날 다시 하게된다면 아래 순서로 해야할 것이다.
1. xml 사진 거르고
2. xml 변환 한 다음에
3. xml 파일명 수정시 filename도 함께 수정.